In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install moviepy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
from moviepy import VideoFileClip
from typing import Dict, List, Optional, Tuple, Union
import os


def create_labeled_video(
    clip: VideoFileClip,
    xs_arr: np.ndarray,
    ys_arr: np.ndarray,
    mask_array: Optional[np.ndarray] = None,
    dotsize: int = 4,
    colormap: str = "cool",
    fps: Optional[float] = None,
    filename: str = "movie.mp4",
    start_time: float = 0.0,
) -> None:
    """Helper function for creating annotated videos.

    Args
        clip
        xs_arr: shape T x n_joints
        ys_arr: shape T x n_joints
        mask_array: shape T x n_joints; timepoints/joints with a False entry will not be plotted
        dotsize: size of marker dot on labeled video
        colormap: matplotlib color map for markers
        fps: None to default to fps of original video
        filename: video file name
        start_time: time (in seconds) of video start

    """

    if mask_array is None:
        mask_array = ~np.isnan(xs_arr)

    n_frames, n_keypoints = xs_arr.shape

    # set colormap for each color
    colors = make_cmap(n_keypoints, cmap=colormap)

    # extract info from clip
    nx, ny = clip.size
    dur = int(clip.duration - clip.start)
    fps_og = clip.fps

    # upsample clip if low resolution; need to do this for dots and text to look nice
    if nx <= 100 or ny <= 100:
        upsample_factor = 2.5
    elif nx <= 192 or ny <= 192:
        upsample_factor = 2
    else:
        upsample_factor = 1

    if upsample_factor > 1:
        clip = clip.resize((upsample_factor * nx, upsample_factor * ny))
        nx, ny = clip.size

    print(f"Duration of video [s]: {np.round(dur, 2)}, recorded at {np.round(fps_og, 2)} fps!")

    def seconds_to_hms(seconds):
        # Convert seconds to a timedelta object
        td = datetime.timedelta(seconds=seconds)

        # Extract hours, minutes, and seconds from the timedelta object
        hours = td // datetime.timedelta(hours=1)
        minutes = (td // datetime.timedelta(minutes=1)) % 60
        seconds = td % datetime.timedelta(minutes=1)

        # Format the hours, minutes, and seconds into a string
        hms_str = f"{hours:02}:{minutes:02}:{seconds.seconds:02}"

        return hms_str

    # add marker to each frame t, where t is in sec
    def add_marker_and_timestamps(get_frame, t):
        image = get_frame(t * 1.0)
        # frame [ny x ny x 3]
        frame = image.copy()
        # convert from sec to indices
        index = int(np.round(t * 1.0 * fps_og))
        # ----------------
        # markers
        # ----------------
        for bpindex in range(n_keypoints):
            if index >= n_frames:
                print("Skipped frame {}, marker {}".format(index, bpindex))
                continue
            if mask_array[index, bpindex]:
                xc = min(int(upsample_factor * xs_arr[index, bpindex]), nx - 1)
                yc = min(int(upsample_factor * ys_arr[index, bpindex]), ny - 1)
                frame = cv2.circle(
                    frame,
                    center=(xc, yc),
                    radius=dotsize,
                    color=colors[bpindex].tolist(),
                    thickness=-1,
                )
        # ----------------
        # timestamps
        # ----------------
        seconds_from_start = t + start_time
        time_from_start = seconds_to_hms(seconds_from_start)
        idx_from_start = int(np.round(seconds_from_start * 1.0 * fps_og))
        text = f"t={time_from_start}, frame={idx_from_start}"
        # define text info
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        font_thickness = 1
        # calculate the size of the text
        text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
        # calculate the position of the text in the upper-left corner
        offset = 6
        text_x = offset  # offset from the left
        text_y = text_size[1] + offset  # offset from the bottom
        # make black rectangle with a small padding of offset / 2 pixels
        cv2.rectangle(
            frame,
            (text_x - int(offset / 2), text_y + int(offset / 2)),
            (text_x + text_size[0] + int(offset / 2), text_y - text_size[1] - int(offset / 2)),
            (0, 0, 0),  # rectangle color
            cv2.FILLED,
        )
        cv2.putText(
            frame,
            text,
            (text_x, text_y),
            font,
            font_scale,
            (255, 255, 255),  # font color
            font_thickness,
            lineType=cv2.LINE_AA,
        )
        return frame

    clip_marked = clip.fl(add_marker_and_timestamps)
    clip_marked.write_videofile(filename, codec="libx264")
    clip_marked.close()


def make_cmap(number_colors: int, cmap: str = "cool"):
    color_class = plt.cm.ScalarMappable(cmap=cmap)
    C = color_class.to_rgba(np.linspace(0, 1, number_colors))
    colors = (C[:, :3] * 255).astype(np.uint8)
    return colors

In [3]:
import subprocess
def extract_clips_ffmpeg_after_reencode(input_video_path, timestamps, clip_length, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for idx, start_time in enumerate(timestamps):
        end_time = start_time + clip_length
        input_basename_ext = os.path.basename(input_video_path)
        input_basename, _ = os.path.splitext(input_basename_ext)
        output_filename = input_basename + f"_clip_{idx+1}_{start_time:.2f}s_to_{end_time:.2f}s.mp4"
        output_path = os.path.join(output_dir, output_filename)

        if os.path.isfile(output_path):
            continue

        command = [
            'ffmpeg',
            '-ss', str(start_time),
            '-i', input_video_path,
            '-t', str(clip_length),
            '-c', 'copy',             # Copy codec (no re-encoding)
            output_path
        ]
        subprocess.run(command, check=True)
        print(f"Clip saved to {output_path}")

In [ ]:
import os
import numpy as np
import pandas as pd
from moviepy.editor import VideoFileClip

def process_and_label_clips(input_video_path, df, timestamps, clip_length, output_dir, labeled_output_dir):
    # Step 1: Extract clips using your provided function
    extract_clips_ffmpeg_after_reencode(input_video_path, timestamps, clip_length, output_dir)
    
    # Ensure output directories exist for labeled videos
    if not os.path.exists(labeled_output_dir):
        os.makedirs(labeled_output_dir)
    
    # Step 2: Process each clip
    for idx, start_time in enumerate(timestamps):
        # Construct clip filename (must match the naming in extract_clips_ffmpeg_after_reencode)
        input_basename = os.path.splitext(os.path.basename(input_video_path))[0]
        clip_filename = os.path.join(
            output_dir, 
            f"{input_basename}_clip_{idx+1}_{start_time:.2f}s_to_{start_time+clip_length:.2f}s.mp4"
        )
        
        if not os.path.isfile(clip_filename):
            print(f"Clip file {clip_filename} not found, skipping...")
            continue
        
        # Step 3: Load the clip using MoviePy
        clip = VideoFileClip(clip_filename)
        
        # Step 4: Filter the DataFrame to get tracking data for this clip
        # Assuming df['time'] is in seconds
        df_clip = df[(df['time'] >= start_time) & (df['time'] < start_time + clip_length)]
        
        # Convert tracking data to NumPy arrays. Here, we assume one keypoint per frame.
        # If you have multiple keypoints per frame, adjust the reshaping accordingly.
        xs_arr = df_clip['x'].to_numpy().reshape(-1, 1)  # shape: (T, 1)
        ys_arr = df_clip['y'].to_numpy().reshape(-1, 1)  # shape: (T, 1)
        
        # Create a mask for valid (non-NaN) values
        mask_array = ~np.isnan(xs_arr)
        
        # Step 5: Define output path for the labeled video
        labeled_filename = os.path.join(
            labeled_output_dir,
            f"labeled_clip_{idx+1}_{start_time:.2f}s_to_{start_time+clip_length:.2f}s.mp4"
        )
        
        # Step 6: Label the clip
        create_labeled_video(
            clip=clip,
            xs_arr=xs_arr,
            ys_arr=ys_arr,
            mask_array=mask_array,
            filename=labeled_filename,
            start_time=start_time
        )
        
        # Close the clip to free resources
        clip.close()


In [6]:
df_test = pd.read_csv('/root/capsule/data/BottomViewPylon1-MIB-2025-02-17/inference/behavior_716325_2024-05-31_10-31-14/bottom_camera.csv')

/tmp/ipykernel_2808/838475983.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('/root/capsule/data/BottomViewPylon1-MIB-2025-02-17/inference/behavior_716325_2024-05-31_10-31-14/bottom_camera.csv')


In [9]:
df_test1 = df_test[2:].reset_index(drop=True)

In [12]:
df_test2 = pd.read_csv('/root/capsule/data/BottomViewPylon1-MIB-2025-02-17/inference/behavior_716325_2024-05-31_10-31-14/bottom_camera.csv',header=[0,1,2],index_col=0)

In [15]:
df_test1.head()

,scorer,heatmap_tracker,heatmap_tracker.1,heatmap_tracker.2,heatmap_tracker.3,heatmap_tracker.4,heatmap_tracker.5,heatmap_tracker.6,heatmap_tracker.7,heatmap_tracker.8,...,heatmap_tracker.23,heatmap_tracker.24,heatmap_tracker.25,heatmap_tracker.26,heatmap_tracker.27,heatmap_tracker.28,heatmap_tracker.29,heatmap_tracker.30,heatmap_tracker.31,heatmap_tracker.32
0,0,515.850830078125,333.6799011230469,0.9987945556640625,335.4886474609375,322.27191162109375,0.999667763710022,355.119140625,266.0167236328125,0.00017966664745472372,...,0.00016898527974262834,355.7112731933594,269.715576171875,0.0001567225845064968,352.0434875488281,250.73495483398438,0.9971842765808105,362.0303955078125,387.4951171875,0.9979580044746399
1,1,516.1701049804688,335.2160949707031,0.9981264472007751,335.70782470703125,322.0995788574219,0.999774158000946,355.1353454589844,265.5116271972656,0.00017797944019548595,...,0.00017053421470336616,355.8164978027344,269.76177978515625,0.00015676190378144383,352.457763671875,250.5872039794922,0.9968073964118958,361.8670654296875,387.4626770019531,0.9985958337783813
2,2,516.1206665039062,335.470458984375,0.9993218183517456,335.676513671875,321.9776306152344,0.9998406767845154,355.12109375,265.642333984375,0.00018064581672661006,...,0.00017063436098396778,355.53289794921875,269.7445373535156,0.00015661929501220584,352.4293518066406,250.6887664794922,0.9966299533843994,361.6092834472656,387.4251403808594,0.9990935325622559
3,3,516.2051391601562,335.7549133300781,0.9991072416305542,335.9415283203125,322.1065979003906,0.9996438026428223,355.2342224121094,265.6594543457031,0.00017980656411964446,...,0.00017180232680402696,355.35760498046875,269.7208557128906,0.0001566487189847976,352.66864013671875,250.59262084960938,0.995743453502655,361.713134765625,387.47637939453125,0.9988307952880859
4,4,516.283203125,335.7440185546875,0.9988868832588196,335.6446533203125,321.9900817871094,0.9998456239700317,355.1673583984375,265.5515441894531,0.00018082704627886415,...,0.00017130440392065793,355.5014343261719,269.7889709472656,0.00015657953917980194,352.4853210449219,250.59580993652344,0.9969454407691956,361.7376708984375,387.55853271484375,0.9988279342651367


In [ ]:
# code from LP to make video
os.makedirs(os.path.dirname('/root/capsule/scratch/clips_for_labeling/labeled/'), exist_ok=True)
# transform df to numpy array

# get LP data

# Load the CSV file into a DataFrame -- my version if needed
# keypoint_dfs = load_keypoints_from_csv('/root/capsule/data/matt_test_DLC_LP_results_20240902/outputs/video_preds2/bottom_camera.csv')
# print(keypoint_dfs.keys())

# version copied from LP scripts from Di
header_rows = [0, 1, 2]
gt_df = pd.read_csv('/root/capsule/data/BottomViewPylon1-MIB-2025-02-17/inference/behavior_716325_2024-05-31_10-31-14/bottom_camera.csv', header=header_rows, index_col=0)

# LP code
keypoints_arr = np.reshape(gt_df.to_numpy(), [gt_df.shape[0], -1, 3])
xs_arr = keypoints_arr[:, :, 0]
ys_arr = keypoints_arr[:, :, 1]
mask_array = keypoints_arr[:, :, 2] > 0.8
# video generation
video_clip = VideoFileClip('/root/capsule/data/matt_test_DLC_LP_results_20240902/video_predictonly/bottom_camera.mp4')
create_labeled_video(
    clip=video_clip,
    xs_arr=xs_arr,
    ys_arr=ys_arr,
    mask_array=mask_array,
    filename='/root/capsule/scratch/bottom_camera_labeled.mp4',
        )

In [9]:
# version copied from LP scripts from Di
header_rows = [0, 1, 2]
gt_df = pd.read_csv('/root/capsule/data/BottomViewPylon1-MIB-2025-02-17/inference/behavior_716325_2024-05-31_10-31-14/bottom_camera.csv', header=header_rows, index_col=0)

# LP code
keypoints_arr = np.reshape(gt_df.to_numpy(), [gt_df.shape[0], -1, 3])
keypoints_arr

array([[[5.15850830e+02, 3.33679901e+02, 9.98794556e-01],
        [3.35488647e+02, 3.22271912e+02, 9.99667764e-01],
        [3.55119141e+02, 2.66016724e+02, 1.79666647e-04],
        ...,
        [3.55711273e+02, 2.69715576e+02, 1.56722585e-04],
        [3.52043488e+02, 2.50734955e+02, 9.97184277e-01],
        [3.62030396e+02, 3.87495117e+02, 9.97958004e-01]],

       [[5.16170105e+02, 3.35216095e+02, 9.98126447e-01],
        [3.35707825e+02, 3.22099579e+02, 9.99774158e-01],
        [3.55135345e+02, 2.65511627e+02, 1.77979440e-04],
        ...,
        [3.55816498e+02, 2.69761780e+02, 1.56761904e-04],
        [3.52457764e+02, 2.50587204e+02, 9.96807396e-01],
        [3.61867065e+02, 3.87462677e+02, 9.98595834e-01]],

       [[5.16120667e+02, 3.35470459e+02, 9.99321818e-01],
        [3.35676514e+02, 3.21977631e+02, 9.99840677e-01],
        [3.55121094e+02, 2.65642334e+02, 1.80645817e-04],
        ...,
        [3.55532898e+02, 2.69744537e+02, 1.56619295e-04],
        [3.52429352e+02, 2.50

In [10]:
np.shape(keypoints_arr)

(2689719, 11, 3)

In [ ]:
def process_and_label_clips(input_video_path, timestamps, clip_length, clip_output_dir, label_output_dir, keypoint_dataframes):
    # Step 1: Extract clips
    extract_clips_ffmpeg_after_reencode(input_video_path, timestamps, clip_length, clip_output_dir)
    
    # For each timestamp/clip
    for idx, start_time in enumerate(timestamps):
        # Construct expected clip filename (should match the naming scheme in your extract function)
        input_basename_ext = os.path.basename(input_video_path)
        input_basename, _ = os.path.splitext(input_basename_ext)
        clip_filename = f"{input_basename}_clip_{idx+1}_{start_time:.2f}s_to_{start_time+clip_length:.2f}s.mp4"
        clip_path = os.path.join(clip_output_dir, clip_filename)
        
        # Load the clip
        clip = VideoFileClip(clip_path)
        
        # Step 2 & 3: Build xs_arr and ys_arr for the clip
        # We assume each dataframe's 'time' column is in seconds relative to the original video.
        xs_list = []
        ys_list = []
        for key, df in keypoint_dataframes.items():
            # Filter the dataframe for the clip’s time window.
            # You might need to adjust tolerance if your times are not perfectly aligned.
            clip_df = df[(df['time'] >= start_time) & (df['time'] < start_time + clip_length)]
            
            # Here, we assume one row per frame. 
            # If the number of rows doesn't match the number of frames in the clip,
            # you could resample or interpolate the keypoint positions.
            xs_list.append(clip_df['x'].to_numpy())
            ys_list.append(clip_df['y'].to_numpy())
        
        # Convert lists to 2D arrays: each column corresponds to a keypoint.
        # (This requires that all keypoint arrays have the same length.)
        xs_arr = np.column_stack(xs_list)
        ys_arr = np.column_stack(ys_list)
        
        # Optional: Verify that xs_arr.shape[0] (number of timepoints) matches expected frame count.
        expected_frames = int(clip.fps * clip.duration)
        if xs_arr.shape[0] != expected_frames:
            print(f"Warning: Number of keypoint frames ({xs_arr.shape[0]}) does not match video frames ({expected_frames}).")
            # You could add interpolation or padding here if needed.
        
        # Step 4: Create labeled video for this clip
        labeled_clip_filename = f"{input_basename}_clip_{idx+1}_{start_time:.2f}s_to_{start_time+clip_length:.2f}s_labeled.mp4"
        labeled_clip_path = os.path.join(label_output_dir, labeled_clip_filename)
        
        create_labeled_video(clip, xs_arr, ys_arr, filename=labeled_clip_path, start_time=start_time)
        clip.close()


In [ ]:
# def segment_movements(df, max_dropped_frames=3):
#     segments = []
#     current_segment = []
#     nan_counter = 0
    
#     for i, row in df.iterrows():
#         if pd.isna(row['x']) or pd.isna(row['y']):  # Object not detected
#             nan_counter += 1
#         else:  # Object detected
#             nan_counter = 0
            
#         if nan_counter <= max_dropped_frames:  # Allowable dropped frames
#             current_segment.append(row)
#         else:
#             if current_segment:  # Save the current segment
#                 segments.append(pd.DataFrame(current_segment))
#                 current_segment = []  # Start a new segment
                
#     # Add last segment if any
#     if current_segment:
#         segments.append(pd.DataFrame(current_segment))
    
#     return segments

# movements = segment_movements(tongue_filtered, max_dropped_frames=3)

In [ ]:

def segment_movements(df, max_dropped_frames=3):
    segments = []
    current_segment = []
    nan_counter = 0
    
    for i, row in df.iterrows():
        if pd.isna(row['x']) or pd.isna(row['y']):  # Object not detected
            nan_counter += 1
        else:  # Object detected
            nan_counter = 0
            
        if nan_counter <= max_dropped_frames:  # Allowable dropped frames
            current_segment.append(row)
        else:
            if current_segment:  # Save the current segment if not empty
                segment_df = pd.DataFrame(current_segment)
                
                # Check if all values (except time) are NaN
                if not segment_df[['x', 'y', 'xv', 'yv', 'v']].isna().all().all():
                    # Trim only leading and trailing NaNs
                    first_valid_idx = segment_df[['x', 'y']].notna().idxmax().min()
                    last_valid_idx = segment_df[['x', 'y']].notna()[::-1].idxmax().min()
                    trimmed_segment_df = segment_df.loc[first_valid_idx:last_valid_idx].reset_index(drop=True)
                    
                    segments.append(trimmed_segment_df)
                
                current_segment = []  # Start a new segment
    
    # Add last segment if any and trim it
    if current_segment:
        segment_df = pd.DataFrame(current_segment)
        
        if not segment_df[['x', 'y', 'xv', 'yv', 'v']].isna().all().all():
            first_valid_idx = segment_df[['x', 'y']].notna().idxmax().min()
            last_valid_idx = segment_df[['x', 'y']].notna()[::-1].idxmax().min()
            trimmed_segment_df = segment_df.loc[first_valid_idx:last_valid_idx].reset_index(drop=True)
            segments.append(trimmed_segment_df)
    
    return segments


movements = segment_movements(tongue_filtered, max_dropped_frames=3)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_segmented_movements_global_time_colored(segments):
    plt.figure(figsize=(10, 6))
    
    # Find global minimum and maximum time for consistent color mapping
    all_times = pd.concat([segment['time'] for segment in segments])
    global_min_time = all_times.min()
    global_max_time = all_times.max()

    for i, segment in enumerate(segments):
        if not segment['x'].empty:
            # Normalize time based on the global min and max
            norm_time = (segment['time'] - global_min_time) / (global_max_time - global_min_time)
            norm_time = segment['time'] - segment['time'][0]
            
            # Mark start point of each segment
            plt.scatter(segment['x'].iloc[0], segment['y'].iloc[0], c = norm_time[0], cmap='YlGnBu_r', s=20, alpha=0.8)

            # Plot each segment with color gradient based on actual time
            plt.scatter(segment['x'], segment['y'], c=norm_time, cmap='YlGnBu_r', s=5,alpha = 0.8)

            
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.colorbar(label='Time (s)')
    plt.title('Segmented Tongue Movements')
    plt.show()

# Example usage
# Call plot_segmented_movements_global_time_colored on the list of segments from segment_movements function
plot_segmented_movements_global_time_colored(movements[0:50])


In [ ]:

# Function to plot original and segmented data with a categorical colormap
def plot_original_and_segmented_data(original_data, segments, xlim=None):
    plt.figure(figsize=(20, 6))

    # Plot original data
    plt.scatter(original_data['time'], original_data['x'], color='gray', s=30, label='Original Data', alpha=0.5)

    # Use 'tab20' colormap for distinct segment colors
    colors = plt.cm.tab20(np.arange(len(segments)) % 20)  # Cycle colors if more than 20 segments

    for i, segment in enumerate(segments):
        plt.scatter(segment['time'], segment['x'], color=colors[i], s=10)
    
    # Labels and legend
    plt.xlabel('Time')
    plt.ylabel('x position (pix)')
    plt.title('Original and Segmented Movements')
    
    if xlim is not None:
        plt.xlim(xlim)
    else:
        plt.xlim([0, segments[-1]['time'].values[-1]])

    plt.ylim([300,400])
    
    plt.show()


# Example usage
# Call plot_original_and_segmented_data with your original data and segments
plot_original_and_segmented_data(tongue_masked, movements[0:70],[0,60])
plot_original_and_segmented_data(tongue_masked, movements[0:70],[49,52])
plot_original_and_segmented_data(tongue_masked, movements[0:70],[50,50.25])


In [ ]:
#old kinematics filter that filtered all the position and velocity together. more principles to filter position sufficiently then derivative after filtering

def kinematics_filter(df, frame_rate=500, cutoff_freq=20, filter_order=8):
    """
    Applies interpolation and low-pass filtering to kinematic data to smooth trajectories and velocities.
    
    Parameters:
    df : pandas.DataFrame
        Input DataFrame containing time-series kinematic data with required columns: ['time', 'x', 'y'].
    frame_rate : int, optional
        Sampling frequency of the data in Hz (default is 500 Hz).
    cutoff_freq : float, optional
        Cutoff frequency for the low-pass Butterworth filter in Hz (default is 20 Hz).
    filter_order : int, optional
        Order of the Butterworth filter (default is 8).

    Returns:
    pandas.DataFrame
        A DataFrame with interpolated and filtered kinematic data, including time, position (x, y),
        velocity components (xv, yv), and speed (v).
    
    Notes:
    - Interpolates missing data points to ensure evenly spaced timestamps.
    - Computes velocity from positional changes over time.
    - Applies a zero-phase low-pass Butterworth filter to smooth kinematic signals.
    - Retains only the originally available (non-NaN) time points in the final output.
    """
    # Ensure the DataFrame has the required columns
    required_columns = ['time', 'x', 'y']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Input DataFrame must contain the columns: {required_columns}")
    
    # Generate new timestamps for interpolation
    t = df['time'].values
    dt = np.diff(t)
    new_ts = []
    for num in range(len(dt)):
        tspace = dt[num] / (1 / frame_rate)
        intgr = int(np.floor(tspace))
        if intgr >= 2:
            new_t = np.linspace(t[num], t[num + 1], intgr)
            new_ts.extend(new_t)
    new_t = np.unique(np.concatenate((t, new_ts)))
    
    # Interpolate missing data points
    x_nonan = df['x'][df['x'].notna()].values
    y_nonan = df['y'][df['y'].notna()].values
    t_nonan = df['time'][df['x'].notna()].values
    
    x = np.interp(new_t, t_nonan, x_nonan)
    y = np.interp(new_t, t_nonan, y_nonan)
    intrp = pd.DataFrame({'time': new_t, 'x': x, 'y': y})
    
    # Compute velocity
    times = intrp['time'].values
    t_diff = np.gradient(times)
    xv = np.gradient(intrp['x'].values) / t_diff
    yv = np.gradient(intrp['y'].values) / t_diff
    v = np.sqrt(xv**2 + yv**2)
    
    intrp['v'] = v
    intrp['xv'] = xv
    intrp['yv'] = yv
    
    # Apply low-pass Butterworth filter
    cutoff = cutoff_freq / (frame_rate / 2)
    b, a = butter(int(filter_order / 2), cutoff)  # Divide filter order by 2 for filtfilt
    filtered_values = filtfilt(b, a, intrp[['x', 'y', 'v', 'xv', 'yv']].values, axis=0)
    
    filtered_df = pd.DataFrame(filtered_values, columns=['x', 'y', 'v', 'xv', 'yv'])
    filtered_df.insert(0, 'time', intrp['time'].values)
    
    # Keep data only at original (non-NaN) timestamps
    df_temp = df.reindex(columns=list(filtered_df.columns.tolist()))
    df_nonan_index = df['time'][df['x'].notna()].index.tolist()
    filtered_df_nonan_index = filtered_df[filtered_df['time'].isin(t_nonan)].index.tolist()
    df_temp.iloc[df_nonan_index] = filtered_df.iloc[filtered_df_nonan_index]
    
    return df_temp.reset_index(drop=True)

In [3]:
import pandas as pd

# Sample time series data
time_series = pd.DataFrame({'time': pd.to_datetime(['2025-03-28 10:00', '2025-03-28 10:15', '2025-03-28 10:30']), 'value': [10, 20, 30]})

# Sample event data
events = pd.DataFrame({'time': pd.to_datetime(['2025-03-28 10:10', '2025-03-28 10:20', '2025-03-28 10:40']), 'event': ['A', 'B', 'C']})

# Merge using merge_asof
merged_df = pd.merge_asof(events, time_series, on='time', direction='forward')

print(merged_df)

                 time event  value
0 2025-03-28 10:10:00     A   20.0
1 2025-03-28 10:20:00     B   30.0
2 2025-03-28 10:40:00     C    NaN
